In [3]:
import pickle
import pandas as pd

In [2]:
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('kmeans.pkl', 'rb') as file:
    kmeans = pickle.load(file)

In [4]:
cluster_ids = {
    0: "Bang for Buck",
    1: "Location Trap",
    2: "Premium Experience"
}
cluster_performance = pd.read_csv('cluster_performance.csv', index_col=0)
def analyze_hotel_performance(service, cleanliness, value, location_rating, sleep_quality, rooms):
    ratings = [service, cleanliness, value, location_rating, sleep_quality, rooms]
    for rating in ratings:
        if rating < 1 or rating > 5:
            raise ValueError("Ratings have to be between 1-5 only.")
    ratings_scaled = scaler.transform([ratings])
    cluster_id = kmeans.predict(ratings_scaled)[0]
    print(f"Hotel belongs to '{cluster_ids[cluster_id]}' cluster.")
    print("Rating vs peers:")
    columns = cluster_performance.columns.tolist()
    peer_ratings = cluster_performance.loc[cluster_id].tolist()
    for column, peer_rating, hotel_rating in zip(columns, peer_ratings, ratings):
        pct_diff = ((hotel_rating - peer_rating) / peer_rating) * 100
        direction = "underperforming" if pct_diff < 0 else "outperforming"
        print(f"{column.upper()}: {direction.upper()} {pct_diff:.1f}% vs peers. (Avg: {peer_rating:.2f})")

In [5]:
ratings = [1,1,1,5,1,1] # good location but sub par everything else
analyze_hotel_performance(*ratings)

Hotel belongs to 'Location Trap' cluster.
Rating vs peers:
SERVICE: UNDERPERFORMING -53.8% vs peers. (Avg: 2.16)
CLEANLINESS: UNDERPERFORMING -50.4% vs peers. (Avg: 2.02)
VALUE: UNDERPERFORMING -53.4% vs peers. (Avg: 2.14)
LOCATION_RATING: OUTPERFORMING 77.5% vs peers. (Avg: 2.82)
SLEEP_QUALITY: UNDERPERFORMING -50.1% vs peers. (Avg: 2.01)
ROOMS: UNDERPERFORMING -44.1% vs peers. (Avg: 1.79)


/home/daniel/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
